In [1]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import csv
from datetime import datetime, timedelta

# Directory containing known faces
KNOWN_FACES_DIR = r"C:\attendance2\images"
TOLERANCE = 0.6  # Recognition tolerance
FRAME_THICKNESS = 3  # Thickness of bounding box
FONT_THICKNESS = 2  # Thickness of text
MODEL = "cnn"  # The model can be "hog" or "cnn" for detection

# Load known faces and their encodings
def load_known_faces():
    known_faces = []
    known_names = []

    for name in os.listdir(KNOWN_FACES_DIR):
        for filename in os.listdir(f"{KNOWN_FACES_DIR}/{name}"):
            image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}/{name}/{filename}")
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
    print("loaded")
    return known_faces, known_names

# Draw bounding boxes and names
def draw_faces(frame, faces, names):
    for (face, name) in zip(faces, names):
        # Draw a rectangle around the face
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), FRAME_THICKNESS)

        # Draw a label with a name below the face
        y_position = face[1] - 15 if face[1] - 15 > 15 else face[1] + 15
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[1] - 35), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (face[0] + 6, y_position), font, 0.8, (255, 255, 255), FONT_THICKNESS)

# Recognize faces from live video and log to CSV
def recognize_faces_from_live_video(known_faces, known_names):
    cap = cv2.VideoCapture(0)
    
    # Open a CSV file for writing
    with open('C:/attendance2/attendance_log.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Timestamp'])  # Header row
        
        last_record_times = {}  # Dictionary to track last record time for each person
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Process every nth frame to reduce the load
            frame_count += 1
            if frame_count % 5 != 0:  # Change the modulo value to adjust the frame rate
                continue

            # Detect faces with RetinaFace
            faces = RetinaFace.detect_faces(frame)
            print("frame recorded")
            # Prepare face encodings for recognition
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = []
            face_names = []

            for face_key in faces.keys():
                facial_area = faces[face_key]["facial_area"]

                # Append detected face locations for recognition
                face_locations.append(facial_area)

                # Convert RetinaFace coordinates to dlib format
                top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
                # Encode the face using face_recognition
                encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

                for encoding in encodings:
                    # Compare with known faces
                    matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
                    name = "Unknown"

                    # Find the shortest distance to a known face
                    face_distances = face_recognition.face_distance(known_faces, encoding)
                    best_match_index = np.argmin(face_distances)
                    if matches[best_match_index]:
                        name = known_names[best_match_index]
                    print(name, "is identified")
                    face_names.append(name)

            # Log detected names and timestamps if 2 minutes have passed
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            for name in face_names:
                if name != "Unknown":
                    if name not in last_record_times or (datetime.now() - last_record_times[name]) >= timedelta(minutes=2):
                        writer.writerow([name, timestamp])
                        last_record_times[name] = datetime.now()  # Update the last record time for this person

            # Draw the results
            draw_faces(frame, face_locations, face_names)

            # Display the frame
            cv2.imshow('Live Video', frame)
            
            # Press 'q' to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Load known faces
    known_faces, known_names = load_known_faces()
    print("Loaded Done")
    print(known_names)
    recognize_faces_from_live_video(known_faces, known_names)



loaded
Loaded Done
['sadhvika', 'shreyesh']
frame recorded
shreyesh is identified
frame recorded
frame recorded
shreyesh is identified
frame recorded
shreyesh is identified
frame recorded
shreyesh is identified
frame recorded
shreyesh is identified
frame recorded
shreyesh is identified
frame recorded
shreyesh is identified
frame recorded
shreyesh is identified
frame recorded
shreyesh is identified
frame recorded
shreyesh is identified
frame recorded
shreyesh is identified
frame recorded
shreyesh is identified
frame recorded
Unknown is identified
frame recorded
Unknown is identified


In [1]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import csv
from datetime import datetime, timedelta

# Directory containing known faces
KNOWN_FACES_DIR = r"C:\attendance2\images"
TOLERANCE = 0.6  # Recognition tolerance
FRAME_THICKNESS = 3  # Thickness of bounding box
FONT_THICKNESS = 2  # Thickness of text
MODEL = "cnn"  # The model can be "hog" or "cnn" for detection

# Load known faces and their encodings
def load_known_faces():
    known_faces = []
    known_names = []

    for name in os.listdir(KNOWN_FACES_DIR):
        for filename in os.listdir(f"{KNOWN_FACES_DIR}/{name}"):
            image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}/{name}/{filename}")
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
    print("loaded")
    return known_faces, known_names

# Draw bounding boxes and names
def draw_faces(frame, faces, names):
    for (face, name) in zip(faces, names):
        # Draw a rectangle around the face
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), FRAME_THICKNESS)

        # Draw a label with a name below the face
        y_position = face[1] - 15 if face[1] - 15 > 15 else face[1] + 15
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[1] - 35), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (face[0] + 6, y_position), font, 0.8, (255, 255, 255), FONT_THICKNESS)

# Recognize faces from live video and log to CSV
def recognize_faces_from_live_video(known_faces, known_names):
    cap = cv2.VideoCapture(0)
    
    # Open a CSV file for writing
    with open('C:/attendance2/attendance_log.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Timestamp'])  # Header row
        
        last_record_times = {}  # Dictionary to track last record time for each person
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Process every nth frame to reduce the load
            frame_count += 1
            if frame_count % 10!= 0:  # Change the modulo value to adjust the frame rate
                continue

            # Detect faces with RetinaFace
            faces = RetinaFace.detect_faces(frame)
            print("frame recorded")
            # Prepare face encodings for recognition
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = []
            face_names = []

            for face_key in faces.keys():
                facial_area = faces[face_key]["facial_area"]

                # Append detected face locations for recognition
                face_locations.append(facial_area)

                # Convert RetinaFace coordinates to dlib format
                top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
                # Encode the face using face_recognition
                encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

                for encoding in encodings:
                    # Compare with known faces
                    matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
                    name = "Unknown"

                    # Find the shortest distance to a known face
                    face_distances = face_recognition.face_distance(known_faces, encoding)
                    best_match_index = np.argmin(face_distances)
                    if matches[best_match_index]:
                        name = known_names[best_match_index]
                    print(name, "is identified")
                    face_names.append(name)

            # Log detected names and timestamps if 2 minutes have passed
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            for name in face_names:
                if name != "Unknown":
                    if name not in last_record_times or (datetime.now() - last_record_times[name]) >= timedelta(minutes=2):
                        writer.writerow([name, timestamp])
                        last_record_times[name] = datetime.now()  # Update the last record time for this person

            # Draw the results
            draw_faces(frame, face_locations, face_names)

            # Display the frame
            cv2.imshow('Live Video', frame)
            
            # Press 'q' to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Load known faces
    known_faces, known_names = load_known_faces()
    print("Loaded Done")
    print(known_names)
    recognize_faces_from_live_video(known_faces, known_names)



loaded
Loaded Done
['akhil', 'sadhvika', 'shreyesh', 'shreyesh', 'srinath']
frame recorded
frame recorded
